# Install 

In [ ]:
! pip install transformers datasets evaluate
! pip install sacrebleu
! pip install googletrans==3.1.0a0
! pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from datasets import load_dataset, DatasetDict, Dataset
import evaluate
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq
from transformers import pipeline
from googletrans import Translator

# Loading Dataset

In [ ]:
dataset = load_dataset("aslg_pc12")
dataset = dataset["train"].train_test_split(train_size=0.8)

# Back Translation

In [5]:
dataset = dataset["train"].train_test_split(train_size=0.8)

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

source_lang = "text"
target_lang = "gloss"


def preprocess_function_backtrans(examples):
    inputs = examples[source_lang]
    targets = examples[target_lang]
    model_inputs = tokenizer(inputs,  text_target=targets, max_length=128, truncation=True)
    return model_inputs

tokenized_data = dataset.map(preprocess_function_backtrans, batched=True)


In [7]:
sacrebleu = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = sacrebleu.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
training_args = Seq2SeqTrainingArguments(
    output_dir="transformer_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,1.127600,0.742242,16.423700,18.208700
2,0.892800,0.571618,21.720700,18.177200
3,0.826900,0.530255,23.175600,18.175400




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=10527, training_loss=1.1417567611046413, metrics={'train_runtime': 1764.5821, 'train_samples_per_second': 95.434, 'train_steps_per_second': 5.966, 'total_flos': 1289437821468672.0, 'train_loss': 1.1417567611046413, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("fuyulinh04/transformer_model")

model = AutoModelForSeq2SeqLM.from_pretrained("fuyulinh04/transformer_model")

In [ ]:
translator = pipeline("translation", model=model, tokenizer = tokenizer)

prediction = []

def testing_data():
  for i in range(0, 8000): # 50% of the old dataset (about 17000 data points)
    text = dataset["train"][i]["text"]
    prediction.append(translator(text))

In [11]:
testing_data()

In [28]:
def merge_dataset():
  for i in range(8000): # Merge into dataset, if duplicate found => not adding
    if(prediction[i] not in dataset["train"]["gloss"]):
      dataset["train"]["gloss"].append(prediction[i])
      dataset["train"]["text"].append(dataset["train"][i]["text"])

merge_dataset() # Generate new augmented dataset

# Training Model

In [ ]:
# dataset.push_to_hub("fuyulinh04/txt_to_gls_dts")

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [ ]:
from datasets import load_dataset
dataset = load_dataset("fuyulinh04/txt_to_gls_dts")

In [32]:
source_lang = "gloss"
target_lang = "text"


def preprocess_function(examples):
    inputs = examples[source_lang]
    targets = examples[target_lang]
    model_inputs = tokenizer(inputs,  text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [ ]:
tokenized_data = dataset.map(preprocess_function, batched=True)

In [34]:
sacrebleu = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = sacrebleu.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

In [36]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="transformer_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,1.195600,0.816066,46.520500,15.691300


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, gloss. If text, gloss are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 14034
  Batch size = 16
Saving model checkpoint to transformer_model/checkpoint-4000
Configuration saved in transformer_model/checkpoint-4000/config.json
Model weights saved in transformer_model/checkpoint-4000/pytorch_model.bin
tokenizer config file saved in transformer_model/checkpoint-4000/tokenizer_config.json
Special tokens file saved in transformer_model/checkpoint-4000/special_tokens_map.json
Deleting older checkpoint [transformer_model/checkpoint-2500] due to args.save_total_limit
Saving model checkpoint to transformer_model/checkpoint-4500
Configuration saved in transformer_model/checkpoint-4500/config.json
Model weights saved in transformer_model/checkpoint-4500/pytor

In [ ]:
# trainer.push_to_hub()

The BLEU score when applying Back Translation is:

# Load Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("junowhite/transformer_model")

model = AutoModelForSeq2SeqLM.from_pretrained("junowhite/transformer_model")

In [ ]:
translators = Translator()

In [ ]:
translator = pipeline("translation", model=model, tokenizer = tokenizer)

prediction = []
answer = []

def testing_data():
  for i in range(7):
    text = dataset["train"][i]["gloss"]
    answer.append(dataset["train"][i]["text"])
    prediction.append(translators.translate(translator(text)[0]["translation_text"]).text)

In [ ]:
testing_data()
print(prediction)
print(answer)